In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import csv
import cv2
import data_aug
import importlib

Run with trunk kernel

Trim dataset for single image set to one x3, y3, z3 set per image

In [21]:
def make_tip_pos_csv(input_csv, output_csv):
    # Check if the output CSV exists and create it with the header if it doesn't
    if not os.path.exists(output_csv):
        header = ['ID', 'x3', 'y3', 'z3', 'img_filename']
        with open(output_csv, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(header)

    # Read the input CSV
    positions_df = pd.read_csv(input_csv)
    
    # Initialize variables to keep track of the last image filename and the last row to write
    last_img_filename = None
    last_output_row = None

    # Process each row in the input CSV
    for index, row in positions_df.iterrows():
        cur_img_filename = row['img_filename']
        img_name = cur_img_filename.removesuffix(".jpg")
        output_row = [row['ID'], row['x3'], row['y3'], row['z3'], cur_img_filename]
        output_row_augmented = [row['ID'], row['x3'], row['y3'], row['z3'], img_name+'_augmented.jpg']

        # If the current image filename is different from the last, write the last output row
        if last_img_filename is not None and last_img_filename != cur_img_filename:
            with open(output_csv, 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(last_output_row) #non-augmented image
                writer.writerow(last_output_row_augmented)# augmented image

        # Update the last image filename and last output row
        last_img_filename = cur_img_filename
        last_output_row = output_row
        last_output_row_augmented = output_row_augmented

    # Write the last row (for the final image in the sequence)
    if last_output_row is not None:
        with open(output_csv, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(last_output_row) # non-augmented image
            writer.writerow(last_output_row_augmented) # augmented image



In [22]:
data_dir = 'data/single'
input_csv = os.path.join(data_dir, 'single_img_regression.csv')  # replace with your actual CSV filename
output_csv = os.path.join(data_dir, 'single_img_regression_single_tip_pos.csv')
make_tip_pos_csv(input_csv, output_csv)

Plot tip position overlaid on images to verify dataset quality

In [61]:
def convert_to_pillow_coords(df, img_width, img_height):
    """
    Convert robot coordinates to Pillow image coordinates.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'x' and 'z' columns in robot coordinates.
    img_width, img_height (int): Dimensions of the Pillow image.

    Returns:
    pd.DataFrame: DataFrame with 'img_x' and 'img_y' columns for Pillow image coordinates.
    """
    buffer = .01 #m

    # Calculate the min and max values for x and z
    x_min, x_max = df['x'].min() - buffer, df['x'].max() + buffer
    z_min, z_max = df['z'].min() - buffer, df['z'].max() + buffer

    # Calculate the scaling factors for x and z coordinates
    x_scale = img_width / (x_max - x_min)
    z_scale = img_height / (z_max - z_min)

    # Calculate the shifts to center the robot's origin within the image
    x_shift = (x_max + x_min) / 2
    z_shift = (z_max + z_min) / 2

    # Convert robot coordinates to Pillow image coordinates
    df['img_x'] = (df['x'] - x_shift) * x_scale + img_width / 2
    df['img_y'] = (df['z'] - z_shift) * z_scale + img_height / 2

    # Invert the y-axis and x-axis to match Pillow's coordinate system (where (0, 0) is top-left)
    df['img_y'] = img_height - df['img_y']
    df['img_x'] = img_width - df['img_x']

    return df[['img_x', 'img_y']]



In [69]:
import matplotlib.pyplot as plt
from PIL import Image

def plot_tip_positions_on_images(data_dir):
    """
    Plot tip positions on images based on coordinates from a CSV file.

    Parameters:
    data_dir (str): Directory containing the CSV file and the images.
    """

    # Load the CSV file
    csv_file = os.path.join(data_dir, 'single_img_regression_single_tip_pos.csv')
    positions_df = pd.read_csv(csv_file)

    # Folder containing images
    image_folder = os.path.join(data_dir, 'images')

    # Convert entire DataFrame coordinates to Pillow image coordinates once
    coords_df = positions_df[['x3', 'z3']].rename(columns={'x3': 'x', 'z3': 'z'})

    # Get image dimensions from the first image
    sample_image_filename = positions_df['img_filename'].iloc[0]
    sample_image_path = os.path.join(image_folder, sample_image_filename)
    with Image.open(sample_image_path) as img:
        img_width, img_height = img.size

    # Convert all coordinates to image coordinates
    img_coords_df = convert_to_pillow_coords(coords_df, img_width, img_height)

    # Add converted coordinates to the original DataFrame
    positions_df['img_x'] = img_coords_df['img_x']
    positions_df['img_y'] = img_coords_df['img_y']

    # Iterate through each row to plot points on the corresponding images
    for index, row in positions_df.iterrows():
        image_filename = row['img_filename']
        image_path = os.path.join(image_folder, image_filename)

        # Open the image
        with Image.open(image_path) as img:
            img_width, img_height = img.size

            # Create a figure with the same dimensions as the image
            fig, ax = plt.subplots(figsize=(img_width / 100, img_height / 100), dpi=100)

            # Plot the image
            ax.imshow(img)

            # Plot the tip position for this specific row
            ax.scatter([row['img_x']], [row['img_y']], color='blue', s=200)  # s is the size of the point

            # Remove axes for a cleaner output
            ax.axis('off')

            # Save the image with the original dimensions
            output_filename = os.path.join(data_dir, f"verification/output_{image_filename}")
            plt.savefig(output_filename, bbox_inches='tight', pad_inches=0)
            plt.close(fig)

            print(f"Plotted tip position on {image_filename} and saved as {output_filename}.")


In [70]:
plot_tip_positions_on_images('data/single')

Plotted tip position on sample_0.jpg and saved as data/single/verification/output_sample_0.jpg.
Plotted tip position on sample_1.jpg and saved as data/single/verification/output_sample_1.jpg.
Plotted tip position on sample_2.jpg and saved as data/single/verification/output_sample_2.jpg.
Plotted tip position on sample_3.jpg and saved as data/single/verification/output_sample_3.jpg.
Plotted tip position on sample_4.jpg and saved as data/single/verification/output_sample_4.jpg.
Plotted tip position on sample_5.jpg and saved as data/single/verification/output_sample_5.jpg.
Plotted tip position on sample_6.jpg and saved as data/single/verification/output_sample_6.jpg.
Plotted tip position on sample_7.jpg and saved as data/single/verification/output_sample_7.jpg.
Plotted tip position on sample_8.jpg and saved as data/single/verification/output_sample_8.jpg.
Plotted tip position on sample_9.jpg and saved as data/single/verification/output_sample_9.jpg.
Plotted tip position on sample_10.jpg an

Augment images

In [17]:
input_dir = 'data/single/raw'
output_dir = 'data/single/augmented'
importlib.reload(data_aug)

for filename in os.listdir(input_dir+"/images"):
    if filename.lower().endswith(('png', 'jpg', 'jpeg')):
        image_path = os.path.join(input_dir+"/images/", filename)
        data_aug.augment_image(image_path, output_dir)

#todo: add augmented image data to csv
